In [ ]:
import os
import cv2
import random
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import urllib.request as urlreq

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

%matplotlib inline

In [ ]:
face_folder = os.path.join("..", "data", "face", "Face")
non_face_folder = os.path.join("..", "data", "other", "Other")

random_state = 42
max_data_size = 2_000
image_shape = (64, 64)

ycrcb_low_range = [0, 133, 77]
ycrcb_high_range = [255, 173, 127]

hsv_low_range = [0, 58, 30]
hsv_high_range = [33, 255, 255]

LBFmodel = "../model_checkpoints/lbfmodel.yaml"
LBFmodel_url = "https://github.com/kurnianggoro/GSOC2017/raw/master/data/lbfmodel.yaml"

random.seed(random_state)

In [ ]:
def load_images_from_folder(folder, label, img_size=image_shape, max_images=None):
    images_paths = [f for f in os.listdir(folder)]
    random.shuffle(images_paths)

    if max_images is not None:
        images_paths = images_paths[:max_images]

    images = []
    for image_path in images_paths:
        full_image_path = os.path.join(folder, image_path)
        img = cv2.imread(full_image_path)
        if img is not None:
            img = cv2.resize(img, img_size)
            images.append(img)

    return np.array(images), np.full(len(images), label)

In [ ]:
if os.path.exists(LBFmodel):
    print("File exists")
else:
    urlreq.urlretrieve(LBFmodel_url, LBFmodel)
    print("File downloaded")

In [ ]:
face_images, face_labels = load_images_from_folder(
    face_folder, label=True, max_images=max_data_size
)

non_face_images, non_face_labels = load_images_from_folder(
    non_face_folder, label=False, max_images=len(face_images)
)

In [ ]:
X = np.vstack((face_images, non_face_images))
y = np.hstack((face_labels, non_face_labels))

indices = np.random.permutation(len(y))
X = X[indices]
y = y[indices]

In [ ]:
landmark_detector = cv2.face.createFacemarkLBF()
landmark_detector.loadModel(LBFmodel)

In [ ]:
def show_sample_result(results, y, show_misclassifications, num_examples=5, title=""):
    samples = []
    indices = np.random.permutation(len(results))
    results = [results[i] for i in indices]
    y = [y[i] for i in indices]
    for i, (face_detected, image, cleaned_mask) in enumerate(results):
        if show_misclassifications and face_detected != y[i]:
            samples.append((image, cleaned_mask))
        elif not show_misclassifications and face_detected == y[i]:
            samples.append((image, cleaned_mask))
        if len(samples) >= num_examples:
            break

    fig, axes = plt.subplots(2, len(samples), figsize=(15, 6))
    for i, (img, mask) in enumerate(samples):
        axes[0, i].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        axes[0, i].set_axis_off()
        axes[1, i].imshow(mask, cmap="gray")
        axes[1, i].set_axis_off()

    fig.suptitle(
        f"{'Misclassifications' if show_misclassifications else 'Correct Classificationss'} {title}"
    )
    plt.tight_layout()
    plt.show()

In [ ]:
def detectAndLocalizeFace(x, useLBF):
    image = x.copy()

    ycrcb_image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    ycrcb_image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    ycrcb_lower = np.array(ycrcb_low_range, dtype=np.uint8)
    ycrcb_upper = np.array(ycrcb_high_range, dtype=np.uint8)
    hsv_lower = np.array(hsv_low_range, dtype=np.uint8)
    hsv_upper = np.array(hsv_high_range, dtype=np.uint8)

    ycrcb_mask = cv2.inRange(ycrcb_image, ycrcb_lower, ycrcb_upper)
    hsv_mask = cv2.inRange(hsv_image, hsv_lower, hsv_upper)
    combined_mask = cv2.bitwise_or(ycrcb_mask, hsv_mask)

    # Apply morphological operations to clean up the mask
    # First Erode then dilate to removes small noise, then dilate then erode to fill small holes
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    cleaned_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_CLOSE, kernel)
    cleaned_mask = cv2.morphologyEx(cleaned_mask, cv2.MORPH_OPEN, kernel)

    contours, _ = cv2.findContours(
        cleaned_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )

    face_detected = False
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if w * h < 500:
            continue

        if not useLBF:
            face_detected = True
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 1)
            continue

        face_rect = np.array([(x, y, w, h)], dtype=np.int32)
        _, landmarks = landmark_detector.fit(image, face_rect)

        if landmarks:
            face_detected = True
            for landmark in landmarks[0]:
                for x_lm, y_lm in landmark:
                    cv2.circle(image, (int(x_lm), int(y_lm)), 2, (0, 255, 0), -1)
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 1)

    return face_detected, image, cleaned_mask

In [ ]:
results = [detectAndLocalizeFace(image, useLBF=False) for image in X]
y_pred = [result[0] for result in results]

print("Accuracy (without LBF):", accuracy_score(y, y_pred))
print("Classification Report (without LBF):\n", classification_report(y, y_pred))

In [ ]:
cm = confusion_matrix(y, y_pred)

plt.figure(figsize=(5, 4))
sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=np.unique(y),
    yticklabels=np.unique(y),
)

plt.title("Confusion Matrix (without LBF)")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

In [ ]:
show_sample_result(
    results, y, show_misclassifications=True, num_examples=5, title="without LBF"
)

show_sample_result(
    results, y, show_misclassifications=False, num_examples=5, title="without LBF"
)

In [ ]:
results = [detectAndLocalizeFace(image, useLBF=True) for image in X]
y_pred = [result[0] for result in results]

print("Accuracy (with LBF):", accuracy_score(y, y_pred))
print("Classification Report (with LBF):\n", classification_report(y, y_pred))

In [ ]:
cm = confusion_matrix(y, y_pred)

plt.figure(figsize=(5, 4))
sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=np.unique(y),
    yticklabels=np.unique(y),
)

plt.title("Confusion Matrix (with LBF)")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

In [ ]:
show_sample_result(
    results, y, show_misclassifications=True, num_examples=5, title="with LBF"
)

show_sample_result(
    results, y, show_misclassifications=False, num_examples=5, title="with LBF"
)

In [ ]:
def classify_image_from_camera():
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Error: Could not access the webcam.")
        return "Error"

    print("Press 'q' to capture an image and 'e' to exit.")
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("Failed to grab frame.")
                break

            cv2.imshow("Camera Feed", frame)
            key = cv2.waitKey(1)
            if key == ord("q"):
                image = frame

                prediction, returned_image, mask = detectAndLocalizeFace(
                    image, useLBF=True
                )
                result = "Face" if prediction else "Non-Face"
                color = (0, 255, 0) if prediction else (0, 0, 255)

                cv2.putText(
                    returned_image,
                    result,
                    (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1,
                    color,
                    2,
                    cv2.LINE_AA,
                )

                cv2.imshow("Processed Image", returned_image)
                cv2.imshow("Masked Image", mask)

            if key == ord("e"):
                break
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        cap.release()
        cv2.destroyAllWindows()

In [ ]:
classify_image_from_camera()